# Finite Fields and the Rijndael Cipher

Richard Masser-Frye 

MATH 187B, Spring 2024

## Review: What is a field?

Lecture 7 has information on fields, but that was a while ago, so here's a review of the basics. A field is a special type of ring, and a ring is a special type of group, so fields are just really special groups. Recall that a ring is a set of numbers that has two operations defined on it: an addition-like operation, denoted +, and a multiplication-like operation, denoted $\cdot$ or \*, where \* must always have the distributive property over +. In a ring, every element must have an additive inverse. The integers are an example of a ring, and the definition of addition and multiplication on the integers is precisely the definition that we're used to.

So how is a field different? A field must have a multiplicative inverse for *every* element, except for 0. The integers are not a field, because the integer 2 (among others) doesn't have a multiplicative inverse (i.e., there's no integer that you can multiply by 2 to get 1). 

#### Sets that are **not** fields:

- Integers
- Integers mod *n* where *n* is composite
- {0}, by fiat
- The cartesian product of two or more rings, even if those rings themselves are fields

#### Sets that are fields:

- Complex numbers
- Real numbers
- Rational numbers
- Integers mod *p* where *p* is prime

## Polynomial Rings

We got a taste of polynomial rings when we learned about NTRU, but didn't really explore them deeply. Understanding polynomial rings will give us some foundation for thinking about fields of prime power order later on. So what is a polynomial ring?

If we have some ring $R$, then we define a *polynomial over the ring* $R$ to be an expression of the form
$$n_0+n_1x+n_2x^2+...+n_dx^d$$
where all $n_i \in R$, and $x$ is treated as an unknown. If $n_d$ is nonzero, then $d$ is the *degree* of the polynomial.

For any ring $R$, the set of polynomials over $R$ is *also* a ring, and it's denoted $R[x]$, or "$R$ adjoin $x$". $R[x]$ is known as a *polynomial ring*.

$x$ is known as the *indeterminate*.

Addition and multiplication of polynomials works the same as it does normally, except that operations on the coefficients are according to the underlying ring; for example, addition and multiplication are different in integer mod rings than over the real numbers. 

Let's see an example of working with polynomial rings in Sagemath:

In [29]:
# Here we make a polynomial ring over the integers (ZZ)

# The second line defines the variable x as the indeterminate
# of the ring

R = ZZ['x']
x = R.gen()
print(R)

# Now that x is defined as the indeterminate of our ring, we can make
# some polynomials with it, and do a bit of polynomial math

poly = (x + 1) * (x + 2)
print(poly)
print(poly^2)

Univariate Polynomial Ring in x over Integer Ring
x^2 + 3*x + 2
x^4 + 6*x^3 + 13*x^2 + 12*x + 4


In [62]:
# Let's try a different underlying ring: integers mod 3

R = IntegerModRing(3)['x']
x = R.gen()
print(R)

poly = (x + 1) * (x + 2)
print(poly)
print(poly^2)

Univariate Polynomial Ring in x over Ring of integers modulo 3
x^2 + 2
x^4 + x^2 + 1


## Polynomial Reduction

Hopefully, you'll have some experience with polynomial reduction; that is, breaking a polynomial down into the product of smaller polynomials. For example, the polynomial over the integers $x^2-x-12$ reduces to $(x-4)(x+3)$. Consider the following:

Let $p(x)$ be a polynomial over some ring $R$. Let $p(x)$ be reducible to $g(x)(x-a)$ where $g(x)$ is some other polynomial over $R$. We call $a$ a *root* of $p(x)$, since if we evaluate $p(a)$ as a function, we see that 
$$p(a)=g(a)(a-a)=g(a)(0)=0$$
The consequence of this is that any polynomial with a root is reducible.

In [64]:
# R is the integers mod 3 adjoin x

# In this ring, x^3 + 2 can be reduced to (x + 2)^3
# (Of course, this is not true over the real numbers)

poly = x^3 + 2
print(poly.factor())

(x + 2)^3


Note that whether a polynomial is reducible and what it reduces to **depends on the underlying ring**; for example, $x^2+1$ is not reducible as an element of $\mathbb{R}[x]$ (over the real numbers, in other words), but as an element of $\mathbb{C}[x]$, it is reducible to $(x+i)(x-i)$.

## What does this have to do with fields?

None of the polynomial rings I've mentioned so far are fields, because none of them have multiplicative inverses for any polynomials with degree 1 or greater. In other words, any polynomials that have an $x$ in them have no inverse.

To make a field out of polynomials, we can do something similar to integer mod rings, where we "mod" by a certain polynomial. What does that mean? As an example, let's use the integers as our underlying ring, so that our un-modded polynomial ring will be $\mathbb{Z}[x]$. Let's "mod" this by the polynomial $x^3$; we call the resulting ring $\mathbb{Z}[x]/(x^3\cdot \mathbb{Z}[x])$. If you're not familiar with this notation, it basically means that two elements are treated as the same if they differ by an element in the set after the slash; and $x^3\cdot \mathbb{Z}[x]$ is the set of every element in $\mathbb{Z}[x]$ multiplied by $x^3$. 

I know this sounds very dense and complicated, but really it's very similar to integer mod rings. As an example, think of $\mathbb{Z}/5\mathbb{Z}$, where 3 and 13 are treated as the same because they differ by 10, which is a multiple of 5. This sort of thing is called a *quotient ring*.

Analogously, in $\mathbb{Z}[x]/(x^3\cdot \mathbb{Z}[x])$, the polynomials $x+2$ and $2x^3+x+2$ would be treated as the same element, because they differ by $2x^3$, which is a multiple of $x^3$. 

In this polynomial quotient ring, $x-1$ and $-x^2-x-1$ are multiplicative inverses because their product is $-x^3+1$, which is equivalent to 1. 

In [32]:
# Let's make the quotient ring described above in Sagemath
# We start with the polynomial ring over the integers

# Because of the way quotient rings are constructed in Sagemath,
# we have to use a different variable name for the indeterminate
# of the underlying ring, so I chose 'y'

Zpoly = ZZ['y']
y = Zpoly.gen()

# Then mod it by y^3

R = Zpoly.quotient(y^3, 'x')
x = R.gen()

# Now we can try the aforementioned examples

print((x + 2) == (2*x^3 + x + 2))

print((x - 1) * (-x^2 - x - 1))

True
1


### But this still isn't a field.

In an integer mod ring $\mathbb{Z}/n\mathbb{Z}$, an element can't have a multiplicative inverse if it shares a factor with $n$. For example, in the integers mod 8, the number 6 doesn't have an inverse, because both 6 and 8 are both divisible by 2. 

**The same is true for polynomial quotient rings.**

In the above example, $\mathbb{Z}[x]/(x^3\cdot \mathbb{Z}[x])$, the polynomial $x^2+x$ has no inverse because both $x^2+x$ and $x^3$ are divisible by $x$. Therefore $\mathbb{Z}[x]/(x^3\cdot \mathbb{Z}[x])$ is not a field.

In order for an integer mod ring $\mathbb{Z}/n\mathbb{Z}$ to be a field, the number $n$ must be prime, because we don't want it to share any factors with any integers less than it. Similarly, in order for a polynomial quotient ring $R[x]/(p(x)\cdot R[x])$ to be a field, the polynomial $p(x)$ must be irreducible over $R$, because if it can be reduced, then it must be a multiple of some lesser polynomial, and that lesser polynomial won't have an inverse.

### So how do we make a field?

First, the underlying ring must be a field. Second, we must mod the polynomial ring by a polynomial that is irreducible over the underlying ring. 

Here's an example: earlier, we noted that $x^2+1$ is irreducible over the real numbers. Since the real numbers form a field, we can mod it by $x^2+1$ and the resulting polynomial quotient ring will also be a field. We can denote this field $\mathbb{R}[x]/((x^2+1)\cdot \mathbb{R}[x])$, but let's call it $E$ for short. What is $E$ like? Let's check it out with Sagemath:

In [36]:
# First, construct a polynomial ring over the reals
Realspoly = RR['y']
y = Realspoly.gen()

# Then, our quotient ring
E = Realspoly.quotient(y^2 + 1, 'x')
x = E.gen()

print((3*x + 5) * (2*x - 1))
print(x * x)
print(x^3)
print((x + 1) * x)

7.00000000000000*x - 11.0000000000000
-1.00000000000000
-x
x - 1.00000000000000


Every element of $E$ takes the form $ax+b$ where $a$ and $b$ are real numbers. There are no terms of degree 2 or greater because in this field, we treat $x^2+1$ as the equivalent of 0, and if we subtract 1 from both sides, we see that $x^2$ is equivalent to $-1$. That means in this field, $x \cdot x = -1$ (as shown above). Wait a minute... this is just complex numbers!

Let's apply this to a finite field, for instance, the integers mod 3. For simplicity, we'll denote the integers mod 3 as $\mathbb{Z}_3$. First, we should make sure that $x^2+1$ is irreducible in $\mathbb{Z}_3$. If it were reducible, it would reduce to two degree-1 polynomials, and as explained earlier, this would mean that it has 1 or 2 roots. Neither 0, nor 1, nor 2 are roots of $x^2+1$ over $\mathbb{Z}_3$, so it has no roots, and thus it can't be reducible.

Like the complex numbers, elements of this field will be of the form $ax+b$ where $a,b\in\mathbb{Z}_3$. That set has 3 elements, so this field has $3^2=9$ elements. Finally, we've arrived at the subject of this presentation: fields of prime power order.

In [88]:
# Constructing the field just described in Sagemath
Zmod3poly = IntegerModRing(3)['y']
y = Zmod3poly.gen()

F = Zmod3poly.quotient(y^2 + 1, 'x')
x = F.gen()

print('Order of this field: ', factor(F.order()))

print(x^2)
print(2 * x^2)
print(2 * (2*x))
print((2*x) * (x + 2))

Order of this field:  3^2
2
1
x
x + 1


### Didn't NTRU use these fields?

Not exactly. NTRU used polynomial rings modded by $x^N-1$. The quotient rings used by NTRU have the very cool property that $x^{k+qN}$ is equivalent to $x^k$, so the exponents behave like elements of a mod ring.

However, such quotient rings can't be fields because $x^N-1$ reduces to $(x-1)(x^{N-1}+x^{N-2}+...+1)$. 

## Another way of thinking about fields of prime power order

We saw earlier that taking $\mathbb{R}$ and modding it by a particular irreducible polynomial gave us a field equivalent to the complex numbers. This demonstrates two ways of creating the complex numbers:

- The quotient ring $\mathbb{R}[x]/((x^2+1)\cdot \mathbb{R}[x])$
- Taking $\mathbb{R}$, and introducing a new element $i$, which is a root of $x^2+1$

It is not a coincidence that both of these produce the same field. In fact, for any field $F$ and polynomial $p(x)$ irreducible in $F[x]$, these two fields will always be equivalent:

- The quotient ring $F[x]/(p(x)\cdot F[x])$
- The field that results from introducing to $F$ a new element, $g$, which is the root of $p(x)$

The latter is written $F[g]$, and $p(x)$ would be called the "minimal polynomial" of $g$.

Let's suppose we want a field with the order $7^3$. We start with the field $\mathbb{Z}_7$, and we want to find a degree-3 polynomial that is irreducible over it. Any reducible degree-3 polynomial would factor down to a degree-1 polynomial and a degree-2 polynomial, so it would have a root. If we cube all the elements of $\mathbb{Z}_7$, we find that no element's cube is 5, therefore $x^3-5$ is irreducible. Let $g$ magically be a root of $x^3-5$. We now have $\mathbb{Z}_7[g]$, a field with exactly $7^3$ elements.

In [87]:
Zmod7poly = IntegerModRing(7)['y']
y = Zmod7poly.gen()

Zmod7_adjoin_g = Zmod7poly.quotient(y^3 - 5, 'x')
x = Zmod7_adjoin_g.gen()

print('Order of this field: ', factor(Zmod7_adjoin_g.order()))

Order of this field:  7^3


## The Rijndael cipher

In 1997, the National Institute of Standards and Technology (NIST) held a contest where mathematicians were asked to devise new symmetric ciphers. (Symmetric ciphers are those which have the same decryption key as encryption key, like the Vigenere cipher.) The winner was a cipher called Rijndael, created by two Belgian mathematicians, Joan Daemen and Vincent Rijmen. As the winner of the contest, Rijndael was adopted as the NIST's new Advanced Encryption Standard (AES). Since then, it has proven resistant to attacks, and remains one of the world's most widely-used symmetric cryptosystems.

#### Why use a symmetric cryptosystem?

Though cryptosystems like RSA and ElGamal offer great utility by being asymmetric, they're impractical for encrypting large quantities of information; they're slow and require enormous amounts of memory. In practice, information is encrypted symmetrically, and only the symmetric key is encrypted asymmetrically.

### How it works

Rijndael is a block cipher, which means that it splits the plaintext up into blocks of a fixed size and treats these blocks as independent of each other. The version of Rijndael adopted as AES uses blocks of 128 bits (16 bytes). In case you don't know, a bit is a **b**inary dig**it**, i.e. a 1 or a 0, and a byte is 8 bits. In the ASCII encoding, one character is one byte, so for a simple .txt file with no special characters, a block would be 16 characters.

Rijndael consists of a number of "rounds" of encryption on the plaintext, where each round (except the last) is the same, and the number of rounds is 10, 12, or 14 depending on the length of the key. Each round, except the last one, consists of these 4 steps:

1. SubBytes
2. ShiftRows
3. MixColumns
4. AddRoundKey

The last round is different in that it lacks the MixColumns step. 

#### Key Expansion
Before encrypting the plaintext, the key must be expanded so that its total number of bits is the block length (128) times the number of rounds plus 1. This allows us to split the key up into a number of chunks, so that each chunk is the same length as the block, and the number of chunks is 1 more than the number of rounds. The details of how this is done are really complicated, so I'm skipping it.

#### SubBytes
Since each byte consists of 8 bits, and each bit can be 0 or 1, the set of all possible bytes has $2^8$ elements. As this is a prime power, there exists a field with this many elements, and in such a field, we can have the coefficients of a polynomial correspond to the bits of a byte. For example, the byte 01001001 would correspond to $x^6+x^3+1$. 

Rijndael uses the field $\mathbb{Z}_2[x]/(p(x)\cdot\mathbb{Z}_2[x])$ where $p(x)=x^8+x^4+x^3+x+1$. This field will be referred to as $GF(2^8)$.

The first step of SubBytes is to represent each byte as a polynomial, then find that polynomial's multiplicative inverse in $GF(2^8)$. 0 doesn't have an inverse, so if the byte is 0 the inversion is skipped and the byte remains 0.

Then, the byte is represented as a vector of 8 bits, and is multiplied by this matrix:
$$\begin{bmatrix} 1&1&1&1&1&0&0&0\\0&1&1&1&1&1&0&0\\0&0&1&1&1&1&1&0\\0&0&0&1&1&1&1&1\\1&0&0&0&1&1&1&1\\1&1&0&0&0&1&1&1\\1&1&1&0&0&0&1&1\\1&1&1&1&0&0&0&1 \end{bmatrix}$$
Followed by a element-wise XOR with the vector $[0,1,1,0,0,0,1,1]$. XOR is a function that takes two bits and returns a 1 if they are different, or a 0 if they are the same.

#### ShiftRows
In this step, the block's 16 bytes are rearranged. The block is represented as a 4x4 grid of bytes, with bytes 1-4 in the leftmost column, bytes 5-8 in the next column, and so on. The second row is shifted 1 to the left, the third row is shifted 2 to the left, and the fourth row is shifted 3 to the left. 

![shiftrows image](./shiftrows.png)

#### MixColumns
In this step, each column of the above grid is treated as a vector of 4 bytes, and is multiplied by this matrix:
$$\begin{bmatrix} 2_B&3_B&1_B&1_B\\1_B&2_B&3_B&1_B\\1_B&1_B&2_B&3_B\\3_B&1_B&1_B&2_B \end{bmatrix}$$
where $1_B$, $2_B$, and $3_B$ represent the bytes 00000001, 00000010, and 00000011 respectively, and bytes are multiplied and added according to $GF(2^8)$.

#### AddRoundKey
Earlier, the key was expanded to a new length of (block length) times (number of rounds + 1). In this step, the entire block undergoes a bitwise XOR with the chunk of the key corresponding to the current round.

This step is also performed before the first round, using the first chunk from the expanded key.

In [97]:
# Let's construct the field used by Rijndael in Sagemath

Zmod2poly = IntegerModRing(2)['y']
y = Zmod2poly.gen()

Rijndael = Zmod2poly.quotient(y^8 + y^4 + y^3 + y + 1, 'x')
x = Rijndael.gen()

print('Order of the Rijndael field:', factor(Rijndael.order()))

Order of the Rijndael field: 2^8


In [90]:
# A function that recreates the first step of SubBytes
# It takes a string as input and prints a string
def invertByte(b):
    
    # convert byte to polynomial
    poly = 0
    for i in range(8):
        if b[7 - i] == '1':
            poly = poly + x^i
            
    # invert the polynomial if it's not 0
    if poly == 0:
        print('00000000')
        return
    else:
        poly = poly^-1
        
    # convert the polynomial back to a string
    s = ''
    for i in range(8):
        s = str(poly[i]) + s
    
    print(s)

In [96]:
invertByte('00000000') # should print 00000000
invertByte('00000001') # should print 00000001 because 1 is its own inverse
invertByte('11111111')
invertByte('10101010')

00000000
00000001
00011100
00010010


## Sources/Acknowledgments

I am indebted to Prof Jon Aycock, who taught me MATH 103B. I heavily referenced my notes from that class while writing the above explanation of finite fields.

For explaining Rijndael, my main source was the book The Design of Rijndael by Joan Daemen and Vincent Rijmen. Kudos to Geisel library for having a copy.

## Exercises